## `ipywidgets`

Using the `ipywidgets` module, an add-on to Jupyter, you can create interactive Python functions.

In [ ]:
import ipywidgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

### Widget Basics

The simplest interface is the function `ipywidgets.interact`, which, in its simplest form, takes a function and makes a widget that lets you change the values of its arguments. Each time an argument is changed, the function is called again.

In [ ]:
lst = ["hello", "it's me", "i was wondering", "if after all these years you'd like to meet"]

def my_function(the_box_is_checked=False, integer=(0, 2), string=lst):
    pass

ipywidgets.interact(my_function);

The type of widget is inferred from the type of the argument. So, because `this_box_is_checked` is a boolean (`True`/`False`), the widget is a checkbox, because `integer` is a tuple, the widget is a slider, and because `string` is a list, the widget is a dropdown menu.

The above example is obviously useless -- `my_function` does nothing with its arguments. Generally, if we want the interaction to do something that the user can see, we need to either `print` or `plot` something.

The example below is a reasonably minimal example of such an interaction.

In [ ]:
def my_function(this_box_is_checked=False, integer=(0, 2), string=lst):
    if this_box_is_checked:
        print(integer*string)

ipywidgets.interact(my_function);

### Widgets for Interactive Plotting

One of the most common uses is for interactive plotting. In that case, we need to use, instead of the typical `%matplotlib inline` magic, the `%matplotlib notebook` magic to get interactive plots. 

In [ ]:
%matplotlib notebook

The examples below show how to create a variety of interactive plots.

In general, you  set up the plot, then draw something on it that you want the user to interact with. Then you create a function that, when it is called, changes the values of the thing you drew.

#### A Line with Interactive Slope

In the example below, we draw a `line` on an `ax`is using `ax.plot`. The function that modifies it is `f` and it changes the slope of the line using `line.set_data`.

In [ ]:
def setup_plot_cartesian(mn, mx):
    """create a figure and axes with the origin in the center
    and the Cartesian x- and y-axes (y=0 and x=0) drawn in.
    """
    fig, ax = plt.subplots()
    ax.set_xlim([mn, mx]); ax.set_ylim([mn, mx])
    plt.hlines(0, 2*mn, 2*mx)
    plt.vlines(0, 2*mn, 2*mx)
    
    return fig, ax

def make_line_plotter():
    """create a function that plots a line with variable slope parameter,
    suitable for use with ipywidgets.interact.
    """
    fig, ax = setup_plot_cartesian(-1, 1)
    
    xs = np.arange(-2, 2, 1)
    line, = ax.plot(xs, xs)
    
    plt.axis("off")
    
    def f(m=0.):
        line.set_data(xs, m*xs)
    
    return f

f = make_line_plotter()
ipywidgets.interact(f, m=ipywidgets.FloatSlider(0, min=-10, max=10));

#### A Line with Interactive Slope and Color

We're not stuck just modifying the data of a plot. There are a variety of `.set_foo` methods that let us set a property `foo` of a plot.

The example below also sets the color. Notice that we just had to add a line calling the `.set_color` method to our previous example.

As an added bonus, we pull our colors from the [XKCD color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/), which has been added to matplotlib. These colors are often more visually appealing than the base matplotlib colors, and they have very memorable names!

In [ ]:
def make_colorful_line_plotter():
    """create a function that plots a line with variable slope parameter,
    and a variable color, suitable for use with ipywidgets.interact.
    """
    
    fig, ax = setup_plot_cartesian(-1, 1)
    xs = np.arange(-2, 2, 1)
    line, = ax.plot(xs, xs, lw=2)
    
    plt.axis("off")
    
    def f(m=0., color="cloudy blue"):
        line.set_data(xs, m*xs)
        line.set_color("xkcd:"+color)
    
    return f

In [ ]:
color_options = [color_key.split(":")[1] for color_key in matplotlib.colors.XKCD_COLORS.keys()]

f = make_colorful_line_plotter()
ipywidgets.interact(f, m=ipywidgets.FloatSlider(0, min=-10, max=10),
                   color=ipywidgets.Dropdown(options=color_options));

#### Bespoke Data Analysis

Now on to a moderately more useful and data-science-y example.

Normally, we fit our data using maximum likelihood. If we're fitting a Gaussian to our data, that means calculating the mean and standard deviation/variance of the data (or, from another perspective, every time we report our data as a mean ± standard deviation, we're fitting a Gaussian to our data).

Alternatively, we could fit our data by hand! The cell below generates an interactive plot with a histogram of the data and a freely modifiable Gaussian probability curve. You can use the sliders to adjust the center and spread of the curve until it seems to fit the data nicely, and then compare it with the `true` mean `mu` and variance `Sigma`.

In [ ]:
def make_gauss_fitter(data, true_mu, true_Sigma):
    """given a dataset with a given mean mu and variance Sigma,
    construct a function that plots a gaussian pdf with variable
    mean and variance over a normed histogram of the data.
    intended for use with ipywidgets.interact.
    """
    
    xs = np.arange(-5*np.sqrt(true_Sigma)+true_mu,
                   5*np.sqrt(true_Sigma)+true_mu,
                   0.01)
    gauss_pdf = lambda xs, mu, sigma: (1/np.sqrt(2*np.pi*sigma))*np.exp((-0.5*(xs-mu)**2)/sigma)
    
    fig, ax = plt.subplots()
    hist = ax.hist(data, density=True, bins=max(10, N//50))
    fit, = plt.plot(xs, gauss_pdf(xs, 0, 1), lw=4) 
    
    def f(mu=0., sigma=1.):
        fit.set_data(xs, gauss_pdf(xs, mu, sigma))
    
    return f

true_mu = 2.; true_Sigma = 0.5; N = 20;
data = np.sqrt(true_Sigma)*np.random.standard_normal(size=N) + true_mu


f = make_gauss_fitter(data, true_mu, true_Sigma)
ipywidgets.interact(f, mu=ipywidgets.FloatSlider(0, min=-10, max=10),
                   sigma=ipywidgets.FloatSlider(1., min=1e-1, max=10, step=1e-1));

In [ ]:
import time as not_time

In [ ]:
%%time
not_time.sleep(5)

In [ ]:
%%timeit
np.random.standard_normal(size=(1000,1000)).dot(np.random.standard_normal(size=(1000,1000)))